Name: Bessa Tamarra

<h2>Assignment 3</h2>

In [6]:
from ucimlrepo import fetch_ucirepo 
soybean= fetch_ucirepo(id=91) 
zoo = fetch_ucirepo(id=111) 
heart_disease = fetch_ucirepo(id=45) 
dermatology = fetch_ucirepo(id=33)
breast_cancer = fetch_ucirepo(id=15)
mushroom = fetch_ucirepo(id=73) 

In [7]:
import pandas as pd
X = soybean.data.features
y = soybean.data.targets 
soybean_df = pd.merge(X, y, left_index=True, right_index=True)

X = zoo.data.features
y = zoo.data.targets 
zoo_df = pd.merge(X, y, left_index=True, right_index=True)

X = heart_disease.data.features
y = heart_disease.data.targets 
heart_disease_df = pd.merge(X, y, left_index=True, right_index=True)

X = dermatology.data.features
y = dermatology.data.targets 
dermatology_df = pd.merge(X, y, left_index=True, right_index=True)

X = breast_cancer.data.features
y = breast_cancer.data.targets 
breast_cancer_df = pd.merge(X, y, left_index=True, right_index=True)

X = mushroom.data.features
y = mushroom.data.targets 
mushroom_df = pd.merge(X, y, left_index=True, right_index=True)

soybean_df = soybean_df.dropna()
zoo_df = zoo_df.dropna()
heart_disease_df = heart_disease_df.dropna()
dermatology_df = dermatology_df.dropna()
breast_cancer_df = breast_cancer_df.dropna()
mushroom_df = mushroom_df.dropna()

In [9]:
def jaccard_coefficient(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union

def ochiai_coefficient(set1, set2):
    intersection = len(set1.intersection(set2))
    denominator = np.sqrt(len(set1) * len(set2))
    return intersection / denominator

def overlap_coefficient(set1, set2):
    intersection = len(set1.intersection(set2))
    min_length = min(len(set1), len(set2))
    return intersection / min_length

def dice_coefficient(set1, set2):
    intersection = len(set1.intersection(set2))
    denominator = len(set1) + len(set2)
    return 2 * intersection / denominator

def graph_based_representation(data):
    num_samples, num_features = data.shape
    similarity_matrix = np.zeros((num_features, num_features))
    for i, j in itertools.combinations(range(num_features), 2):
        similarity_matrix[i, j] = jaccard_coefficient(set(data[:, i]), set(data[:, j]))
        similarity_matrix[j, i] = similarity_matrix[i, j]
    G = nx.from_numpy_array(similarity_matrix)
    embedding = SpectralEmbedding(n_components=p)
    representation_matrix = embedding.fit_transform(similarity_matrix)
    return representation_matrix

def joint_operation(data, representation_matrix):
    return np.dot(data, representation_matrix)

def mean_operation(data, representation_matrix):
    return np.mean(np.dot(data, representation_matrix), axis=1)

def perform_clustering(data, k):
    kmeans = KMeans(n_clusters=k)
    return kmeans.fit_predict(data)

In [10]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.manifold import SpectralEmbedding
from sklearn.metrics.cluster import adjusted_rand_score, normalized_mutual_info_score, fowlkes_mallows_score
from sklearn.cluster import KMeans
import numpy as np
import networkx as nx
import warnings
import itertools
p = 10
q = 10 
k = 3

results = []
datasets = ["soybean_df", "zoo_df", "heart_disease_df", "dermatology_df", "breast_cancer_df", "mushroom_df"]
for dataset_name in datasets:
    dataset = globals()[dataset_name]
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=UserWarning)
        try:
            X = dataset
            enc = OneHotEncoder()
            X_encoded = enc.fit_transform(X)
            representation_matrix = graph_based_representation(X_encoded.toarray())
            integrated_data = joint_operation(X_encoded.toarray(), representation_matrix)
            labels = perform_clustering(integrated_data, k)

            true_labels = dataset.iloc[:, -1] 
            ARI = adjusted_rand_score(true_labels, labels)
            NMI = normalized_mutual_info_score(true_labels, labels)
            FMI = fowlkes_mallows_score(true_labels, labels)
            results.append([dataset_name, ARI, NMI, FMI])
        except UserWarning as e:
            print(f"Warning: {e}")

results_df = pd.DataFrame(results, columns=["Dataset", "ARI", "NMI", "FMI"])
print(results_df)

C:\Users\bessa\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\bessa\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\bessa\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\bessa\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\clus

            Dataset       ARI       NMI       FMI
0        soybean_df  0.677392  0.774817  0.785382
1            zoo_df  0.534283  0.715232  0.715982
2  heart_disease_df  0.001782  0.015164  0.342046
3    dermatology_df  0.192335  0.315293  0.420680
4  breast_cancer_df  0.359508  0.263296  0.667522
5       mushroom_df  0.306460  0.375637  0.648071


C:\Users\bessa\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


For clustering analysis, ARI, NMI and FMI are used to assess clustering performance. ARI corrects for chance and is useful for comparing clusterings with different sizes, especially when a reference clustering is available. NMI measures the shared information between two clusterings, giving a value between 0 and 1 where 1 indicates identical clusterings. It's good for comparing clusterings without a reference and handling varying cluster sizes. FMI assesses clustering similarity by combining precision and recall, making it valuable when both are important for evaluation. 

<h4>Kmodes and Hierarchical Clustering, perform categorical data clustering, use FMI, ARI, and NMI for the comparison of performance </h4>

In [12]:
from sklearn.preprocessing import LabelEncoder 
from kmodes.kmodes import KModes
from sklearn.cluster import AgglomerativeClustering

results_categorical = []

for dataset_name in datasets:
    dataset = globals()[dataset_name]
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=UserWarning)
        try:
            X_cat = dataset.iloc[:, :-1]
            true_labels_cat = dataset.iloc[:, -1]

            encoder = LabelEncoder()
            X_cat_encoded = X_cat.apply(encoder.fit_transform)

            km = KModes(n_clusters=k, init='Huang', n_init=5, verbose=0)
            km_labels = km.fit_predict(X_cat_encoded)
            ARI_km = adjusted_rand_score(true_labels_cat, km_labels)
            NMI_km = normalized_mutual_info_score(true_labels_cat, km_labels)
            FMI_km = fowlkes_mallows_score(true_labels_cat, km_labels)

            ac = AgglomerativeClustering(n_clusters=k, linkage='ward')
            ac_labels = ac.fit_predict(X_cat_encoded)
            ARI_ac = adjusted_rand_score(true_labels_cat, ac_labels)
            NMI_ac = normalized_mutual_info_score(true_labels_cat, ac_labels)
            FMI_ac = fowlkes_mallows_score(true_labels_cat, ac_labels)
            
            results_categorical.append([dataset_name + " (Kmodes)", ARI_km, NMI_km, FMI_km])
            results_categorical.append([dataset_name + " (Hierarchical)", ARI_ac, NMI_ac, FMI_ac])
        except UserWarning as e:
            print(f"Warning: {e}")

results_categorical_df = pd.DataFrame(results_categorical, columns=["Dataset", "ARI", "NMI", "FMI"])
print(results_categorical_df)

                            Dataset       ARI       NMI       FMI
0               soybean_df (Kmodes)  0.675984  0.710181  0.774863
1         soybean_df (Hierarchical)  0.653689  0.837666  0.783908
2                   zoo_df (Kmodes)  0.721570  0.714510  0.812035
3             zoo_df (Hierarchical)  0.461701  0.585056  0.645736
4         heart_disease_df (Kmodes)  0.194250  0.175247  0.487704
5   heart_disease_df (Hierarchical)  0.009543  0.010944  0.353434
6           dermatology_df (Kmodes)  0.515638  0.679720  0.677660
7     dermatology_df (Hierarchical)  0.032201  0.078147  0.293614
8         breast_cancer_df (Kmodes)  0.408917  0.429897  0.683847
9   breast_cancer_df (Hierarchical)  0.781678  0.688725  0.893555
10             mushroom_df (Kmodes)  0.313601  0.367445  0.647829
11       mushroom_df (Hierarchical)  0.298722  0.431275  0.610515


$$
\text{The ARI, NMI, and FMI metrics are importanting for evaluating clustering performance. These metrics are selected because of their ability to quantify the quality of clustering results. FMI typically yields higher scores compared to ARI and NMI because it prioritizes precise classification. ARI's tendency to favor clustering with more clusters and NMI's sensitivity to both cluster number and size can result in inaccurate assessments of clustering effectiveness.} 
$$

$$
\text{In contrast, FMI places emphasis on accurate classification, making it a more dependable metric for evaluating clustering quality and thus often surpassing ARI and NMI in performance evaluations across different contexts.}
$$